In [180]:
# Imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn


In [181]:
# Import data
df = pd.read_csv('data/train.csv')
# Reduce dataframe the number of lines by half
df = df.sample(frac=0.1)

In [182]:
#import the test data
df_test = pd.read_csv('data/test.csv')

In [183]:
# Convert country code 'UK' to 'GB'
df['country'] = df['country'].replace('UK', 'GB')
df_test['country'] = df_test['country'].replace('UK', 'GB')

In [184]:
# Function that converts the timestamp to a datetime object
def convert_timestamp(timestamp):
    date = pd.to_datetime(timestamp, unit='s')
    # extract the hour from the timestamp
    hour = date.hour
    return hour

In [185]:
# Convert the timestamp to a datetime object
df['time'] = df['timeStamp'].apply(convert_timestamp)
df_test['time'] = df_test['timeStamp'].apply(convert_timestamp)

In [186]:
# For each country, create a individual datatframe with the mean of the sold ads for each hour

# Concatenate time and country columns to create a new dataframe
df['time_country'] = df['country'].astype(str) + '_' + df['time'].astype(str)
df_test['time_country'] = df_test['country'].astype(str) + '_' + df_test['time'].astype(str)

In [187]:
df['opeartingSystem'] = df['opeartingSystem'].str.replace('macOS','Mac OS')
df_test['opeartingSystem'] = df_test['opeartingSystem'].str.replace('macOS','Mac OS')
df['opeartingSystem'] = df['opeartingSystem'].str.replace('OS X','Mac OS')
df_test['opeartingSystem'] = df_test['opeartingSystem'].str.replace('OS X','Mac OS')


In [188]:
# get timezone for each country
from datetime import datetime, tzinfo
from dateutil import tz
import pytz
# Create a function that returns the local time for a gevin country
def get_local_time(country, time):
    dt_str = pd.to_datetime(time, unit='s').strftime('%m/%d/%Y %H:%M:%S')
    format = "%m/%d/%Y %H:%M:%S"
    # Create datetime object in local timezone
    dt = datetime.strptime(dt_str, format)
    # Create datetime object in UTC timezone
    dt_utc = dt.replace(tzinfo=pytz.UTC)
    # Create datetime object in country timezone
    try:
        country_tz = pytz.country_timezones(country)[0]
        dt_country = dt_utc.astimezone(pytz.timezone(country_tz))
        return dt_country.strftime('%H')
    except:
        return pd.NaT


In [189]:
# Create a new column with the local time for each country
#df['local_time'] = df.apply(lambda row: get_local_time(row['country'], row['timeStamp']), axis=1)


In [190]:
# Function that returns 'neutral' if the value is between 02 and 08
def neutral(hour):
    if hour == '02':
        return 'neutral'
    elif hour == '03':
        return 'neutral'
    elif hour == '04':
        return 'neutral'
    elif hour == '05':
        return 'neutral'
    elif hour == '06':
        return 'neutral'
    elif hour == '07':
        return 'neutral'    
    elif hour == '08':
        return 'neutral'
    else:
        return hour

        # Apply the function to the column 'local_time'
#df['local_time'] = df['local_time'].apply(neutral)

        # Function that returns 'bad' if the value is between 18 and 23


def bad(hour):
    if hour == '18':
        return 'bad'
    elif hour == '19':
        return 'bad'
    elif hour == '20':
        return 'bad'
    elif hour == '21':
        return 'bad'
    elif hour == '22':
        return 'bad'
    elif hour == '23':
        return 'bad'
    else:
        return hour

# Apply the function to the column 'local_time'
#df['local_time'] = df['local_time'].apply(bad)


In [191]:
# function that converts number to text
def convert_number(number):
    if number == 1:
        return 'one'
    elif number == 2:
        return 'two'
    else:
        return 'other'

# Apply the function to the column 'integrationType'
df['integrationType'] = df['integrationType'].apply(convert_number)
df_test['integrationType'] = df_test['integrationType'].apply(convert_number)

In [192]:
# Function that returns 'other' if the country is not in the list: ['US', 'GB', 'ES', 'FR', 'DE', 'IT', 'CH', 'AU', 'MX', 'BR', 'CA', 'JP']
def country_other(country):
    if country not in ['US', 'GB', 'ES', 'FR', 'DE', 'IT', 'AU', 'MX', 'BR', 'CA', 'JP']:
        return 'other'
    else:
        return country

# Apply the function to the column 'country'
df['country_prep'] = df['country'].apply(country_other)
df_test['country_prep'] = df_test['country'].apply(country_other)


In [193]:
count_websiteId = df['websiteId'].value_counts()
count_websiteId_test = df_test['websiteId'].value_counts()

count_websiteId.sort_values(ascending=False)
count_websiteId_test.sort_values(ascending=False)

top_websiteId = count_websiteId.index[:11]
top_websiteId_test = count_websiteId_test.index[:11]

top_websiteId=top_websiteId.tolist()
top_websiteId_test=top_websiteId_test.tolist()

def prefix(list, prefix):

    rename_list= list
    for i in range(len((list))):

        rename_list[i]= prefix + str(list[i])
    return rename_list

top_websiteId_prefix = prefix(top_websiteId, 'websiteId_')
top_websiteId_test_prefix = prefix(top_websiteId_test, 'websiteId_')
df['websiteId'] = df['websiteId'].apply(lambda x: 'websiteId_' + str(x) if 'websiteId_' + str(x) in top_websiteId_prefix else 'other_websiteId')
df_test['websiteId'] = df_test['websiteId'].apply(lambda x: 'websiteId_' + str(x) if 'websiteId_' + str(x) in top_websiteId_test_prefix else 'other_websiteId')

In [194]:
# count the number of each countries in the dataframe

count_browser = df['browser'].value_counts()
count_browser_test = df_test['browser'].value_counts()
count_browser.sort_values(ascending=False)
count_browser_test.sort_values(ascending=False)
top_browser = count_browser.index[:11]
top_browser_test = count_browser_test.index[:11]

df['browser'] = df['browser'].apply(lambda x: x if x in top_browser else 'other_browser')
df_test['browser'] = df_test['browser'].apply(lambda x: x if x in top_browser_test else 'other_browser')

In [195]:
# same for operatingSystem
count_opeartingSystem = df['opeartingSystem'].value_counts()
count_opeartingSystem_test = df_test['opeartingSystem'].value_counts()
count_opeartingSystem.sort_values(ascending=False)
count_opeartingSystem_test.sort_values(ascending=False)
top_opeartingSystem = count_opeartingSystem.index[:8]
top_opeartingSystem_test = count_opeartingSystem_test.index[:8]

df['opeartingSystem'] = df['opeartingSystem'].apply(lambda x: x if x in top_opeartingSystem else 'other_opeartingSystem')
df_test['opeartingSystem'] = df_test['opeartingSystem'].apply(lambda x: x if x in top_opeartingSystem_test else 'other_opeartingSystem')

In [196]:
# same for environmentType
count_environmentType = df['environmentType'].value_counts()
count_environmentType_test = df_test['environmentType'].value_counts()
count_environmentType.sort_values(ascending=False)
count_environmentType_test.sort_values(ascending=False)
top_environmentType = count_environmentType.index[:11]
top_environmentType_test = count_environmentType_test.index[:11]

df['environmentType'] = df['environmentType'].apply(lambda x: x if x in top_environmentType else 'other_environmentType')
df_test['environmentType'] = df_test['environmentType'].apply(lambda x: x if x in top_environmentType_test else 'other_environmentType')


In [197]:
# same for device
count_device = df['device'].value_counts()
count_device_test = df_test['device'].value_counts()
count_device.sort_values(ascending=False)
count_device_test.sort_values(ascending=False)
top_device = count_device.index[:11]
top_device_test = count_device_test.index[:11]

df['device'] = df['device'].apply(lambda x: x if x in top_device else 'other_device')
df_test['device'] = df_test['device'].apply(lambda x: x if x in top_device_test else 'other_device')

In [198]:
# définir ceux qui apparaissent le plus
count_placementId = df['placementId'].value_counts()
count_placementId_test = df_test['placementId'].value_counts()

# keep 10 bests
best_placementId = count_placementId[:6]
best_placementId_test = count_placementId_test[:6]

# print(best_placementId)
best_placementId_index = best_placementId.index.tolist()
best_placementId_test_index = best_placementId_test.index.tolist()
# print(best_placementId_index)
# rename
prefixPId = 'PId_'

#PId_renames = [(prefixPId + sub) for sub in best_placementId_index]

def rename(list, prefix):
    rename_list = list
    for i in range(len((list))):
        rename_list[i] = prefix + str(list[i])
    return rename_list

Pid_rename = rename(best_placementId_index, prefixPId)
Pid_test_rename = rename(best_placementId_test_index, prefixPId)
# send back data

df2 = pd.DataFrame()

df2['placementId'] = df['placementId'].apply(
    lambda x: prefixPId + str(x) if prefixPId + str(x) in Pid_rename else 'PId_Other')

df2_test = pd.DataFrame()
df2_test['placementId'] = df_test['placementId'].apply(
    lambda x: prefixPId + str(x) if prefixPId + str(x) in Pid_test_rename else 'PId_Other')
    


In [199]:
# définir ceux qui apparaissent le plus
count_deepthree = df['hashedRefererDeepThree'].value_counts()
count_deepthree_test = df_test['hashedRefererDeepThree'].value_counts()
count_deepthree
count_deepthree_test

# keep 10 bests
best_deepthree = count_deepthree[:10]
best_deepthree_test = count_deepthree_test[:10]


# print(best_placementId)
best_deepthree_index = best_deepthree.index.tolist()
best_deepthree_test_index = best_deepthree_test.index.tolist()

print(best_placementId_index)
# rename
prefixdeepthree = 'deepthree_'


#PId_renames = [(prefixPId + sub) for sub in best_placementId_index]

def rename(list, prefix):
    rename_list = list
    for i in range(len((list))):
        rename_list[i] = prefix + str(list[i])
    return rename_list


deepthree_rename = rename(best_deepthree_index, prefixdeepthree)
deepthree_test_rename = rename(best_deepthree_test_index, prefixdeepthree)
# send back data
df3 = pd.DataFrame()


df3['hashedRefererDeepThree'] = df['hashedRefererDeepThree'].apply(
    lambda x: prefixdeepthree + str(x) if prefixdeepthree + str(x) in deepthree_rename else 'deepthree_Other')

df3_test = pd.DataFrame()
df3_test['hashedRefererDeepThree'] = df_test['hashedRefererDeepThree'].apply(
    lambda x: prefixdeepthree + str(x) if prefixdeepthree + str(x) in deepthree_test_rename else 'deepthree_Other')



['PId_121842', 'PId_121840', 'PId_121836', 'PId_108383', 'PId_117831', 'PId_91837', 'PId_123180', 'PId_121895', 'PId_113907', 'PId_129333']


In [200]:
# get_dumy for the column 'local_time'
# df_dumy_time = pd.get_dummies(df['local_time'])
# add get_dumy for the column 'device'
df_dumy_device = pd.get_dummies(df['device'])
# add get_dumy for the column 'integraionType' and change the name of the columns
df_dumy_integrationType = pd.get_dummies(df['integrationType'])
# add get_dumy for the column 'articleSafenessCategorization'
df_dumy_articleSafenessCategorization = pd.get_dummies(df['articleSafenessCategorization'])
# add get_dumy for the column 'opeartingSystem'
df_dumy_opeartingSystem = pd.get_dummies(df['opeartingSystem'])
# add get_dumy for the column 'websiteId'
df_dumy_websiteId = pd.get_dummies(df['websiteId'])
# add get_dumy for the column 'browser'
df_dumy_browser = pd.get_dummies(df['browser'])
# add get_dumy for the column 'placementId'
df_dumy_placementId = pd.get_dummies(df2['placementId'])
# add get_dumy for the column 'hashedRefererDeepThree'
df_dumy_hashedRefererDeepThree = pd.get_dummies(df3['hashedRefererDeepThree'])
# add get_dumy for the column 'country'
df_dumy_country = pd.get_dummies(df['country_prep'])

# Concatenate all the dataframes
df_dumy = pd.concat([
    # df_dumy_time,
    df_dumy_device,
    df_dumy_integrationType,
    df_dumy_articleSafenessCategorization,
    df_dumy_opeartingSystem,
    df_dumy_websiteId,
    df_dumy_browser,
    df_dumy_placementId,
    df_dumy_hashedRefererDeepThree,
    df_dumy_country
], axis=1)

# Apply the same operation to the test dataframe
# df_dumy_time_test = pd.get_dummies(df_test['local_time'])
# add get_dumy for the column 'device'
df_dumy_device_test = pd.get_dummies(df_test['device'])
# add get_dumy for the column 'integraionType' and change the name of the columns
df_dumy_integrationType_test = pd.get_dummies(df_test['integrationType'])
# add get_dumy for the column 'articleSafenessCategorization'
df_dumy_articleSafenessCategorization_test = pd.get_dummies(df_test['articleSafenessCategorization'])
# add get_dumy for the column 'opeartingSystem'
df_dumy_opeartingSystem_test = pd.get_dummies(df_test['opeartingSystem'])
# add get_dumy for the column 'websiteId'
df_dumy_websiteId_test = pd.get_dummies(df_test['websiteId'])
# add get_dumy for the column 'browser'
df_dumy_browser_test = pd.get_dummies(df_test['browser'])
# add get_dumy for the column 'placementId'
df_dumy_placementId_test = pd.get_dummies(df2_test['placementId'])
# add get_dumy for the column 'hashedRefererDeepThree'
df_dumy_hashedRefererDeepThree_test = pd.get_dummies(df3_test['hashedRefererDeepThree'])
# add get_dumy for the column 'country'
df_dumy_country_test = pd.get_dummies(df_test['country_prep'])

# Concatenate all the dataframes
df_dumy_test = pd.concat([
    # df_dumy_time_test,
    df_dumy_device_test,
    df_dumy_integrationType_test,
    df_dumy_articleSafenessCategorization_test,
    df_dumy_opeartingSystem_test,
    df_dumy_websiteId_test,
    df_dumy_browser_test,
    df_dumy_placementId_test,
    df_dumy_hashedRefererDeepThree_test,
    df_dumy_country_test
], axis=1)



In [201]:
df_dumy.head()
df_dumy.columns

Index(['IPTV', 'PC', 'Phone', 'Tablet', 'one', 'two', 'safe', 'uncat',
       'unsafe', 'Android', 'Chrome OS', 'Fire OS', 'Linux', 'Mac OS',
       'Windows', 'iOS', 'other_opeartingSystem', 'unknown', 'other_websiteId',
       'websiteId_20808', 'websiteId_41456', 'websiteId_42779',
       'websiteId_48060', 'websiteId_48177', 'websiteId_48915',
       'websiteId_48916', 'websiteId_53583', 'websiteId_57369',
       'websiteId_68951', 'websiteId_69655', 'Android webview', 'Chrome',
       'Chrome Mobile', 'Facebook App', 'Firefox', 'Google App',
       'Microsoft Edge', 'Mobile Samsung Browser', 'Pinterest App', 'Safari',
       'Safari mobile', 'other_browser', 'PId_108383', 'PId_113907',
       'PId_117831', 'PId_121836', 'PId_121840', 'PId_121842', 'PId_121895',
       'PId_123180', 'PId_129333', 'PId_91837', 'PId_Other',
       'deepthree_18f3e539d6760fbb391d94194bb850916080e90db29f32bbccffa2fc8db248e6',
       'deepthree_25c75b8c9e13fd262ecb525f8e678a959781f0960583f32951759dd504d

In [202]:
df_dumy_test.head()
# show all the different columns
df_dumy_test.columns

Index(['IPTV', 'PC', 'Phone', 'Tablet', 'one', 'two', 'safe', 'uncat',
       'unsafe', 'Android', 'Chrome OS', 'Fire OS', 'Linux', 'Mac OS',
       'Windows', 'iOS', 'other_opeartingSystem', 'unknown', 'other_websiteId',
       'websiteId_20808', 'websiteId_41456', 'websiteId_42779',
       'websiteId_48060', 'websiteId_48177', 'websiteId_48915',
       'websiteId_48916', 'websiteId_53583', 'websiteId_57369',
       'websiteId_68951', 'websiteId_69655', 'Android webview', 'Chrome',
       'Chrome Mobile', 'Facebook App', 'Firefox', 'Google App',
       'Microsoft Edge', 'Mobile Samsung Browser', 'Pinterest App', 'Safari',
       'Safari mobile', 'other_browser', 'PId_108383', 'PId_112035',
       'PId_117831', 'PId_121836', 'PId_121840', 'PId_121842', 'PId_121895',
       'PId_123180', 'PId_129333', 'PId_91837', 'PId_Other',
       'deepthree_18f3e539d6760fbb391d94194bb850916080e90db29f32bbccffa2fc8db248e6',
       'deepthree_25c75b8c9e13fd262ecb525f8e678a959781f0960583f32951759dd504d

In [203]:
#compare the columns of the train and test dataframes
for col in df_dumy.columns:
    if col not in df_dumy_test.columns:
        print(col)


PId_113907


In [204]:
# Make a logistic regression model
from sklearn.linear_model import LogisticRegression

In [205]:
# Fit the model with multithreading
model = LogisticRegression(max_iter=1000, n_jobs=-1)
model.fit(df_dumy, df['isSold'])

LogisticRegression(max_iter=1000, n_jobs=-1)

In [206]:
# Show the wheights of the model
model.coef_

array([[-0.65290111,  0.33675634,  0.25065275,  0.04768857,  0.12320237,
        -0.14100581,  0.4319493 ,  0.18436495, -0.63411769,  0.78023583,
        -0.31606329,  1.03168322, -0.31862515,  0.0633329 , -0.10516516,
         0.21908127, -0.21183705, -1.16044601,  0.6829805 , -0.33701566,
         0.36391616,  0.03084667, -0.3092692 ,  0.44149614, -0.26564287,
        -0.32462272, -2.21235041, -0.24298386,  1.61482819,  0.54001362,
        -0.40761954,  0.31624887,  0.28281539,  0.21844466, -0.23592151,
         0.54747679, -0.27134319,  0.24541004, -0.03708775, -0.43140453,
        -0.11969753, -0.12512515, -0.24298386,  1.25575834, -0.26564287,
        -1.5234997 ,  0.1429798 , -1.33815826,  1.68653649,  0.54001362,
         0.91935563, -2.21235041,  1.02018779,  0.79961139, -0.44558761,
        -0.20249456, -0.47984957,  0.68313938,  0.64483624,  0.2908552 ,
         0.03084667, -1.28132699, -0.41887123,  0.36103763, -0.02581084,
         0.06003628, -0.21874482, -0.09899141,  0.0

In [207]:
# Make a prediction on the test data
df_test['isSold'] = model.predict(df_dumy_test)

/home/zakenobi/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- PId_112035
Feature names seen at fit time, yet now missing:
- PId_113907




In [208]:
pred = model.predict(df_dumy_test)
pred

/home/zakenobi/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- PId_112035
Feature names seen at fit time, yet now missing:
- PId_113907




array([False,  True,  True, ...,  True, False, False])

In [209]:
# apply the same changes to the test data
df_test.head()

,auctionId,timeStamp,placementId,websiteId,hashedRefererDeepThree,country,opeartingSystem,browser,browserVersion,device,environmentType,integrationType,articleSafenessCategorization,time,time_country,country_prep,isSold
0,00277acc-dee5-4065-a580-c49dbde5c02e_6c6fd7b0-...,1604180182,120706,other_websiteId,8448d2635d8ce25f3758907451fc740c4b2131693c7a8b...,CA,iOS,Facebook App,NaN,Phone,js-web,two,safe,21,CA_21,CA,False
1,0055d24f-bbf0-418d-84e7-df1d3124d6d6_60eaa750-...,1604145756,120706,other_websiteId,513dc346ab569ee76d6dbc927079b4049b1e9e95833d13...,US,Android,Facebook App,293_0,Phone,js-web,two,safe,12,US_12,US,True
2,005bc881-b650-4ccc-baa6-850c9710b0ea_15181501-...,1604095938,120706,other_websiteId,7b856dd2cffe043b51f48f9094a6a8bc58dbdea3bdd239...,MX,Android,Facebook App,274_0,Phone,js-web,two,safe,22,MX_22,MX,True
3,008c5f32-c29f-4726-8315-a2a4f2d10fb2_a4a0167f-...,1604257787,120706,other_websiteId,8448d2635d8ce25f3758907451fc740c4b2131693c7a8b...,US,iOS,Safari mobile,13_1,Phone,js-web,two,safe,19,US_19,US,True
4,00f96be1-59f2-43b6-a6d1-0668001cd08c_1014d516-...,1604265857,120706,other_websiteId,a64b4cf5e410623d0bb3d110b2d3183c1590f69e0e61af...,CA,Windows,Microsoft Edge,86_0,PC,js-web,two,unsafe,21,CA_21,CA,False


In [210]:
# save only the column 'isSold' and the column 'auctionId' to a csv
df_test[['auctionId', 'isSold']].to_csv('prediction.csv', index=False)

In [211]:
df_test[['auctionId', 'isSold']]

,auctionId,isSold
0,00277acc-dee5-4065-a580-c49dbde5c02e_6c6fd7b0-...,False
1,0055d24f-bbf0-418d-84e7-df1d3124d6d6_60eaa750-...,True
2,005bc881-b650-4ccc-baa6-850c9710b0ea_15181501-...,True
3,008c5f32-c29f-4726-8315-a2a4f2d10fb2_a4a0167f-...,True
4,00f96be1-59f2-43b6-a6d1-0668001cd08c_1014d516-...,False
...,...,...
242166,fed928d7-a4a4-4119-9a24-cd1791a581b2,True
242167,fef95305-7ada-4be6-b571-0baa9632c987,True
242168,ff2703c2-5e85-4c68-a136-cfbfe56b16b7,True
242169,ffc29802-20e1-4011-aea1-f00699e8de20,False


In [212]:
len(df_test)

242171

In [213]:
len(pred)

242171

In [214]:
# check if there is nan 
df_test.isnull().sum()

auctionId                            0
timeStamp                            0
placementId                          0
websiteId                            0
hashedRefererDeepThree            3860
country                            108
opeartingSystem                      0
browser                              0
browserVersion                   32387
device                               0
environmentType                      0
integrationType                      0
articleSafenessCategorization        0
time                                 0
time_country                         0
country_prep                         0
isSold                               0
dtype: int64